# Application BI: Aide à la décision
Enzo GUENY  
Adam SERGHINI  

*Sujet choisi: **Sujet 5** - Théorie du choix social*  
Trois profils de vote sont fournis. Le premier comporte **1000** électeurs et **9** candidats, les deux
autres comportent **10 000** électeurs et **12** candidats. Les données (accessibles vu la taille
uniquement en .csv) donnent **l’ordre de préférence** de chaque électeur pour les 12 candidats,
numérotés de 1 à 12.  
L'objectif est de programmer et de comparer les résultats obtenus par différents modes de scrutin possibles (vus en cours ou obtenus par recherche bibliographique).  
  
Nous effectuons les tests sur profil1, profil2, profil3 fournis en consigne, profil4 est un jeu de données extrait du TD *"examen 2"* fait en cours et nous sert de corpus de validation afin de comparer les résultats de nos implémentations avec les résultats obtenus en présentiel.  
Nous implémenterons les méthodes en Python.

In [6]:
import pandas as pd # Used to load data
import numpy as np # Used to make operations on data matrices
from collections import Counter
from itertools import permutations # Used by Keminy-Young
 
CORPUS_SIZE = 4 # How many files are executed

## Methods

### Tools
Méthodes utiles à l'implémentation des algorithmes de choix social

In [73]:
def eliminate(matrix, candidate):
    shape = matrix.shape
    transpose = np.transpose(matrix)
    transpose = transpose[transpose != candidate]
    transpose = transpose.reshape(shape[1], shape[0] - 1)
    return np.transpose(transpose)

def getCandidates(matrix):
    return np.unique(matrix)

def executeDuels(matrix):
    candidates = getCandidates(matrix)
    shape = matrix.shape
    results = np.zeros((candidates.size, candidates.size), int)
    for candidate in candidates:
        for adversary in candidates:
            if(adversary != candidate):
                win = 0
                for i in range(0, shape[1]):
                    for j in range(0, shape[0]):
                        if(matrix[j][i] == adversary):
                            break
                        if(matrix[j][i] == candidate):
                            win = win +1
                            break

                results[candidate-1][adversary-1] = win
    return results

def getKey(item):
    return item[1]

### Implementation
Implémentation des algorithmes de choix social vus en cours. Le principe de chaque méthode est décrit en commentaire dans le code source.

In [100]:
# Selects elected candidate based on first preference
def relativeMajority(matrix, returnSize):
    return Counter(matrix[0]).most_common(returnSize)[0]

# Selects two winners for a first turn using relativeMajority()
# Eliminates every other candidate then processes a second turn among remaining candidates
def relativeMajorityT2(matrix):
    candidates = getCandidates(matrix)
    winners = [Counter(matrix[0]).most_common(2)[0][0], Counter(matrix[0]).most_common(2)[1][0]]
    eliminated = set(candidates) - set(winners)
    new_matrix = np.copy(matrix)
    for loser in eliminated:
        new_matrix[(new_matrix == loser)] = -1
        new_matrix = eliminate(new_matrix, -1)
    return Counter(new_matrix[0]).most_common(1)[0]
    
# Selects elected candidate based on borda method
# Score is calculated by adding (n - preference_order) for each voter
def borda(matrix):
    n = matrix.shape[0]
    scores = np.zeros(n).astype(np.int)
    voters = matrix.shape[1]
    for i in range (0, voters-1):
        for j in range (0, n-1):
            scores[matrix[j][i] -1] += (n - j)
    return scores.argmax()+1, scores.max()

# Returns the candidate winning the most duels against others
def condorcet(matrix):
    candidates = getCandidates(matrix)
    results = executeDuels(matrix)
    n = matrix.shape[1]
    winners = []
    w_duels = 0
    for candidate in candidates:
        duels = Counter(x >= n/2 for x in results[candidate-1])
        if(duels[True] > w_duels):
            winners = []
            w_duels = duels[True]
        if(w_duels == duels[True]):
            winners = np.append(winners, candidate)

    return winners, w_duels

def kemenyYoung(matrix):
    candidates = getCandidates(matrix)
    best_score = 0
    best_ranking = None
    ketcha = executeDuels(matrix)
    for candidates_ranking in permutations(candidates):
        score = 0
        for candidate in range (0, candidates.size):
            for adversary in range (candidate, candidates.size): 
                score += ketcha[candidates_ranking[candidate]-1][candidates_ranking[adversary]-1]
        if score > best_score:
            best_score, best_ranking = score, candidates_ranking
    return (best_score, best_ranking)


# Eliminates the candidate being the least preffered each iteration
# until 2 remains, returns the most popular
def coombs(matrix):
    n = matrix.shape[0] -1
    new_matrix = np.copy(matrix)
    for i in range (0, n-1):
        unpopular =  Counter(new_matrix[new_matrix.shape[0] - 1]).most_common(1)[0][0]
        new_matrix = eliminate(new_matrix, unpopular)
    return Counter(new_matrix[0]).most_common(1)[0]

# Eliminates the candidate being the least top-voted each iteration
# until 2 remains, returns the most popular
def alternative(matrix):
    n = matrix.shape[0] -1
    new_matrix = np.copy(matrix)
    for i in range (0, n-1):
        unpopular =  Counter(new_matrix[0]).most_common(new_matrix.shape[0])[new_matrix.shape[0]-1][0]
        new_matrix = eliminate(new_matrix, unpopular)
    return Counter(new_matrix[0]).most_common(1)[0]

# Returns the candidate with the best score won on duelsagainst others minus lost duels
def copeland(matrix):
    candidates = getCandidates(matrix)
    results = executeDuels(matrix)
    n = matrix.shape[1]

    winners = []
    w_duels = 0
    for candidate in candidates:
        duels = Counter(x >= n/2 for x in results[candidate-1]) - Counter(x < n/2 for x in results[candidate-1])
        if(duels[True] > w_duels):
            winners = []
            w_duels = duels[True]
        if(w_duels == duels[True]):
            winners = np.append(winners, candidate)

    return winners, w_duels
    return 0


## Data

In [9]:
# Load Data
# data/choix_social/

matrices = [None] * CORPUS_SIZE # Stores matrix conversions of each file
for i in range (0, CORPUS_SIZE):
    filename = 'data/choix_social/profil'+ str(i+1) + '.csv'
    data = pd.read_csv(filename, header=None)
    matrices[i] = data.to_numpy()
    print("Profil" + str(i+1) + ": " + str(matrices[i].shape))

Profil1: (9, 1000)
Profil2: (12, 10000)
Profil3: (12, 10000)
Profil4: (4, 100)


||Profil1|Profil2|Profil3|Profil4|
|:-|:-:|:-:|:-:|:-:|
|Nombre de candidats|9|12|12|4|
|Nombre de votants|1,000|10,000|10,000|100|

## Execution

In [ ]:
#Relative Majority 1 turn
print("Relative Majority 1 turn:")
for i in range (0, CORPUS_SIZE):
    result = relativeMajority(matrices[i], 1)
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")

#Relative Majority 2 turns
print("\nRelative Majority 2 turns:")
for i in range (0, CORPUS_SIZE):
    result = relativeMajorityT2(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")

# Condorcet
print("\nCondorcet :")
for i in range (0, CORPUS_SIZE):
    result = condorcet(matrices[i])
    print("Profil" + str(i+1) + ": Candidats " + str(result[0]) + ": " + str(result[1]) + " duels gagnés")


# Borda
print("\nBorda :")
for i in range (0, CORPUS_SIZE):
    result = borda(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + "pts")

# Coombs
print("\nCoombs :")
for i in range (0, CORPUS_SIZE):
    result = coombs(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix reportées (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")

# Alternative
print("\nAlternative :")
for i in range (0, CORPUS_SIZE):
    result = alternative(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix reportées (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")

# Copeland
print("\nCopeland :")
for i in range (0, CORPUS_SIZE):
    result = copeland(matrices[i])
    print("Profil" + str(i+1) + ": Candidats " + str(result[0]) + ": " + str(result[1]) + " duels gagnés (- duels perdus)")

# KemenyYoung
print("\nKemeny Young :")
for i in range(0, CORPUS_SIZE):
    result = kemenyYoung(matrices[i])
    print("Profil"+ str(i+1) + ": Classement" + str(result[1]) + ": " + str(result[0]) + "pts")




Relative Majority 1 turn:
Profil1: Candidat 4: 293 voix (29.30 %)
Profil2: Candidat 10: 1828 voix (18.28 %)
Profil3: Candidat 7: 1498 voix (14.98 %)
Profil4: Candidat 1: 35 voix (35.00 %)

Relative Majority 2 turns:
Profil1: Candidat 4: 564 voix (56.40 %)
Profil2: Candidat 5: 6366 voix (63.66 %)
Profil3: Candidat 7: 5078 voix (50.78 %)
Profil4: Candidat 4: 65 voix (65.00 %)

Condorcet :
Profil1: Candidats [5.]: 8 duels gagnés
Profil2: Candidats [7.]: 11 duels gagnés
Profil3: Candidats [ 7.  8. 11.]: 10 duels gagnés
Profil4: Candidats [3.]: 3 duels gagnés

Borda :
Profil1: Candidat 5: 6451pts
Profil2: Candidat 7: 95904pts
Profil3: Candidat 6: 80155pts
Profil4: Candidat 3: 306pts

Coombs :
Profil1: Candidat 5: 524 voix reportées (52.40 %)
Profil2: Candidat 7: 5398 voix reportées (53.98 %)
Profil3: Candidat 11: 5712 voix reportées (57.12 %)
Profil4: Candidat 3: 75 voix reportées (75.00 %)

Alternative :
Profil1: Candidat 4: 564 voix reportées (56.40 %)
Profil2: Candidat 7: 5398 voix repor

Profil2: Classement(7, 5, 8, 10, 4, 12, 2, 3, 1, 6, 11, 9): 491309pts


Profil3: Classement(8, 7, 11, 6, 5, 1, 12, 4, 9, 2, 10, 3): 443696pts
Profil4: Classement(3, 2, 4, 1): 410pts


## Méthode supplémentaire

## Synthèse des Résultats et interprétation
### Profil 1
|Majorité Relative (1 tour)|Majorité Relative (2 tours)|Condorcet|Borda|Coombs|Kemeny-Young|Alternative|Copeland|
|:-|:-|:-|:-|:-|:-|:-|:-|
|**4** (293 - *29.30%*)|**4** (564 - *56.40%*)|**5** (8)|**5** (6,451 pts)|**5** (524 - *52.40%*)|{5, 4, 7, 8, 6, 3, 9, 2, 1} (24597 pts)  |**4** (564 - *56.40%*)|**5** (7)|
### Profil 2
|Majorité Relative (1 tour)|Majorité Relative (2 tours)|Condorcet|Borda|Coombs|Kemeny-Young|Alternative|Copeland|
|:-|:-|:-|:-|:-|:-|:-|
|**10** (1828 - *18.28%*)|**5** (6366 - *63.66%*)|**7** (11)|**7** (95,904 pts)|**7** (5398 - *53.98%*)|{7, 5, 8, 10, 4, 12, 2, 3, 1, 6, 11, 9} (491309pts)|**7** (5398 - *53.98%*)|**7** (10)|
### Profil 3
|Majorité Relative (1 tour)|Majorité Relative (2 tours)|Condorcet|Borda|Coombs|Kemeny-Young|Alternative|Copeland|
|:-|:-|:-|:-|:-|:-|:-|
|**7** (1498 - *14.97%*)|**7** (5078 - *50.78%*)|**7 8 11** (10)|**6** (80,155 pts)|**5** (5712 - *57.12%*)|{8, 7, 11, 6, 5, 1, 12, 4, 9, 2, 10, 3} (443696pts)|**6** (5010 - *50.10%*)|**7 8 11** (8)|
    Comme on peut le voir sur les résultats ci-dessus, toutes les méthodes ne retournent pas toujours les mêmes vainqueurs.
    Tout d'abord, pour le profil 1, les candidats 4 et 5 s'échangent la première place sur toutes les méthodes, cependant les méthodes de Condorcet et de Borda étant plus évoluées que les votes à majorité relative, le candidat 5 semble être le plus indiqué.
    Pour les profils 2 et 3, on peut observer un pattern assez similaire dans la mesure où il n'y pas de vainqueur récurrent pour toutes les méthodes. 
    Néanmoins, la méthode de Condorcet ne fournit pas de résultat clair car il existe des cycles au sein des résultats, notamment pour le profil 2 ou les candidats 7,8 et 11 sont en concurrence direct pour être le gagnant de Condorcet. Ici, la méthode de Copeland peut permettre d'éclaircir ces résultats, toutefois il peut rester des cycles.
    Enfin, la méthode de Kemeny-Young permet de désambiguïser les résultats de Condorcet en proposant un score par classement pour clarifier les résultats issus de Condorcet. La méthode de Kemeny Young est dite Condorcet-cohérente car elle fournira toujours le même résultats que la méthode de Condorcet (lorsqu'il existe).
    Pour le profil 2, le candidat 7 semble être le plus indiqué, pour le profil 3 les résultats sont moins équilibrés, il est donc plus ardu d'en sortir un vainqueur global.

## Systèmes éléctoraux

Il n'existe pas de méthode parfaite pour déterminer le gagnant d'un vote. Nous avons vu dans nos expérimentations que chaque méthode est susceptible de donner une issue
différente en s'attaquant à une problématique donnée mais impliquant de nouvelles injustices.

### Majorité relative
Nous pouvons sans difficulté constater que la méthode de **Majorité relative à 1 tour** est la pire possible lorsque plus de deux candidats sont en
compétition. En effet son issue dépend d'une proportion de la population classant un candidat en premier. Dans nos études de cas en TP et en projet
nous avons souvent fait face à une situation dans laquelle une minorité significative classait un candidat en premier choix tandis que la majorité
(le reste de la population) classait ce candidat en dernier qui pourtant l'emportait.  
Dans une élection réelle, on peut supposer qu'il s'agisse d'un parti extrêmiste avec une base d'électeurs importante mais minoritaire, et dont la 
majorité de la population n'adhère pas à leur idéologie.  
  
Dans un système de suffrage universel direct à deux tours comme en France, cette injustice est corrigée par un deuxième tour opposant les deux candidats les 
plus populaires et ainsi donner la possibilité à la population d'affirmer leur volonté de NE PAS vouloir du candidat arrivé en tête au premier tour.

![](https://cdn-europe1.lanmedia.fr/var/europe1/storage/images/europe1/politique/presidentielle-les-resultats-du-premier-tour-3308975/40761974-38-fre-FR/Presidentielle-2017-les-resultats-en-integralite-compares-aux-estimations-des-instituts.jpg)

Ci-dessus les résultats au premier tour des élections présidentielles de 2017 en France.  
Ce système implique cependant de nouveaux problèmes de justice comme le **vote stratégique**. En effet il est rarement intéréssant d'être honnête sur son
classement personnel des candidats car vous risquez de donner votre unique voix à un candidat qui sera à coup sur éliminé au premier tour. Une solution est
donc le vote stratégique: voter pour un candidat populaire que l'on apprécie moins mais cependant plus proche de son idéologie que ses adversaires.  
Ce système favorise également les extrèmes, s'il existe plusieurs candidats d'idéologie proche alors ils devront se répartir les voix de leur électorat
tandis qu'un parti extrémiste n'aura aucun concurrent et conservera la totalité de son électorat, lui donnant alors un meilleur résultat. C'est pour cette raion par exemple
que François Bayrou du MoDem a décidé de se retirer de l'élection de 2017, risquant de faire du tort à Emmanuel Macron et par conséquent favoriser Marine LePen.

### Vote par classement

Les méthodes de **Condorcet**, **Vote Alternatif** etc. sont des méthodes de vote pour lesquelles l'électeur doit renseigner un ordre de préférence. Ce 
système permet d'éviter le vote stratégique car il invite les électeurs à être le plus honnête possible, étant donné que leurs choix secondaires seront également
pris en compte pour déterminer le vainqueur. Certains pays comme L'*Irlande* et Le *Sri Lanka* procèdent par vote alternatif ou "Second tour instantané" pour élire
leur président. Ce système a aussi des défauts notament l'effet de **Center squeeze** qui a pour effet d'éliminer les candidats idéologiquement modérés étant
au "centre" de l'échiquier politique et favorise les candidats ayant une idéologie plus tranchée.  

![](https://static.miraheze.org/electowikiwiki/1/1a/5-candidate_center_squeeze_IRV_animation.gif)

### Conclusion

Somme toute faite, toutes les méthodes possèdent des points forts et des points faibles (qui sont détaillées ci-dessus), en définitive **il n'existe pas du tout de système assurant la cohérence** (Théorème d'impossibilité d'Arrow).  
Ce travail nous a permis de comprendre plus en profondeur comment fonctionnent les différents systèmes électoraux, leurs avantages et inconvénients. Il a attisé notre
curiosité politique en nous offrant un regard critique sur les différentes facons de contabiliser les voix.  
Ce projet s'inscrit également dans un contexte historique d'élections américaines sans précédent.

## Sources et références

*Youtube: Primer - Simulating alternate voting systems* - https://www.youtube.com/watch?v=yhO6jfHPFQU  
*ElectroWiki - Center-Squeeze* - https://electowiki.org/wiki/Center_squeeze  
*Wikipedia - Vote Alternatif* - https://fr.wikipedia.org/wiki/Vote_alternatif  
*Théorème d'impossibilité d'Arrow* - https://fr.wikipedia.org/wiki/Th%C3%A9or%C3%A8me_d%27impossibilit%C3%A9_d%27Arrow